# NN2: Implementacja propagacji wstecznej błędu
Adrianna Grudzień 

### Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

### Wczytanie danych

In [2]:
sq_train = pd.read_csv("../data/regression/square-simple-training.csv", index_col=0)
sq_test = pd.read_csv("../data/regression/square-simple-test.csv", index_col=0)
x = np.matrix(sq_train['x'])
y_true = np.matrix(sq_test['y'])
# x = x[:10]
# y_true = y_true[:10]

### Funkcja aktywacji

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return np.exp(-x) / (1 + np.exp(-x))**2

### Definicja sieci MLP

-`np.multiply` mnoży wektory po każdym elemencie

In [4]:
class MLP:
    def __init__(self, input_layer_size=[100,1], layers_sizes=[90,3,1], activ_fun=sigmoid):
        self.input_layer_size = input_layer_size
        self.layers_sizes = layers_sizes # wszystkie warstwy poza pierwszą
        self.activ_fun = activ_fun
        self.number_of_layers = len(layers_sizes)+1 # liczba wszystkich warstw
        self._initialize_weights_and_biases()

#         for w in self.biases:
#             print('biases: ', w.shape)
#         for w in self.weights:
#             print('weights: ', w.shape)
        
    def _initialize_weights_and_biases(self, min_val=-0.5, max_val=0.5):
        self.weights = [np.random.uniform(min_val, max_val, 
                                          size=[self.layers_sizes[0], self.input_layer_size[1]])]
        self.biases = []   
        
        for i in range(len(self.layers_sizes)-1):
            self.weights.append(np.random.uniform(min_val, max_val, 
                                                  size=[self.layers_sizes[i+1], self.layers_sizes[i]]))
            self.biases.append(np.random.uniform(min_val, max_val, size=(self.layers_sizes[i],1)))
            
        self.biases.append(np.random.uniform(min_val, max_val, 
                                             size=(self.layers_sizes[len(self.layers_sizes)-1],1)))
        
    @staticmethod # wywołuje się bezpośrednio na obiekcie
    def mse(y_pred, y_true):
        """Funkcja kosztu C."""
        return np.mean(np.square(y_pred-y_true))
    @staticmethod
    def sigmoid_derivative(x):
        return np.exp(-x) / np.square(1 + np.exp(-x))
        
    def forward(self, x):
        self.active = [x] # wektor wartości aktywowanych - a^l
        self.inactive = [x] # nieaktywowane - z^l
        for i in range(len(self.weights)):
            z = self.weights[i] @ self.active[i] + self.biases[i]
            a = self.activ_fun(z)
            self.inactive.append(z)
            self.active.append(a)
        return self.active[-1]
    
    def backpropagate(self, x, y_pred):
        """y_pred ~= aL"""
        delta_biases = [np.zeros(b.shape) for b in self.biases]
        delta_weights = [np.zeros(w.shape) for w in self.weights]
        
        delta = np.multiply(self.active[-1] - y_pred, MLP.sigmoid_derivative(self.inactive[-1])) # dla ostatniej warstwy - L
        delta_biases[-1] = delta
        delta_weights[-1] = delta @ self.active[-2].transpose()
        for l in range(self.number_of_layers-2, 0, -1): #dla warstw (L-1,...,0)
#             print(delta.transpose().shape, self.weights[l].shape, MLP.sigmoid_derivative(self.inactive[l]).transpose().shape)
            delta = np.multiply((delta.transpose() @ self.weights[l]).transpose(), MLP.sigmoid_derivative(self.inactive[l]))
            delta_biases[l-1] = delta
#             print(self.active[l-1].shape, delta.shape)
            delta_weights[l-1] = self.active[l-1] @ delta.transpose()
        
        return (delta_biases, delta_weights)
    
    def print_shapes(self, list1):
        for w in list1:
            print(w.shape)

    def fit(self, x, y_true, n_epochs=1):
        """x, y_true - pojedyncze obserwacje"""
        y_pred = self.forward(x)
        (delta_biases, delta_weights) = self.backpropagate(x, y_pred)
            
        for w, d_w in zip(delta_biases, delta_weights):
            print('b: ', w.shape, 'w: ', d_w.shape)
#         print('melo')
#         self.print_shapes(delta_biases)
#         print('elo')
#         self.print_shapes(delta_weights)
#         self.print_shapes(self.biases)

#         for i in range(n_epochs):
#             y_pred = self.forward(x)
#             (delta_biases, delta_weights) = self.backpropagate(y_pred, y_true)
#             self.print_shapes(self.biases)
#             self.print_shapes(delta_biases)
#             self.print_shapes(self.weights)
#             self.print_shapes(delta_weights)

#             self.biases += delta_biases
#             self.weights += delta_weights


In [5]:
mlp = MLP(layers_sizes=[3,2,1])
mlp.fit(x, y_true)

b:  (3, 100) w:  (1, 3)
b:  (2, 100) w:  (3, 2)
b:  (1, 100) w:  (1, 2)
